# Supply Chain Analytics in Python
- Linear programming is a tool for optimization
- Optimization methods use a mathematical model whose requirements are linar relationships
- 3 components to linear programming
    - decision variables (you control)
    - objective function (math expression that uses variables to express goal)
    - constraints (math expression that describes the limits of solutions)
- PuLP is a library for linear and integer programming problems written in Python
    - It interfaces with solvers e.g. CPLEX, COIN, Gurobi
    
### Simple resource scheduling exercise
In this exercise you are planning the production at a glass manufacturer. This manufacturer only produces wine and beer glasses:

- there is a maximum production capacity of 60 hours
- each batch of wine and beer glasses takes 6 and 5 hours respectively
- the warehouse has a maximum capacity of 150 rack spaces
- each batch of the wine and beer glasses takes 10 and 20 spaces respectively
- the production equipment can only make full batches, no partial batches
- Also, we only have orders for 6 batches of wine glasses. Therefore, we do not want to produce more than this. Each batch of the wine glasses earns a profit of \\$5 and the beer \\$4.5.
- The objective is to maximize the profit for the manufacturer.
- puLP has already been imported for you.

In [ ]:
from pulp import *

In [18]:
# Initialize Class
model = LpProblem("Maximize Glass Co. Profits", LpMaximize)

# Define Decision Variables
wine = LpVariable('Wine', lowBound=0, upBound=None, cat='Integer')
beer = LpVariable('Beer', lowBound=0, upBound=None, cat='Integer')

# Define Objective Function
model += 5 * wine + 4.5 * beer

# Define Constraints
model += 6 * wine + 5 * beer <= 60
model += 10 * wine + 20 * beer <= 150
model += wine <= 6

# Solve Model
model.solve()
print("Produce {} batches of wine glasses".format(wine.varValue))

Produce 6.0 batches of wine glasses


### Trying to optimise calories burned
- 0.2 minutes per pushup = 3 calories per pushup
- 10 minutes per mile = 130 calories
- 10 mins to exercise

- decision variables: num pushups and miles run
- Objective function that uses variables to express goal: max (3 * num pushups + 130 * num miles)
- constraints - limits of solution: 
- 0.2 * num pushups + 10 * num miles ≤ 10
- num pushups ≥ 0
- num miles ≥ 0

In [19]:
model = LpProblem("Maximize Calories Burned", LpMaximize)

# Define Decision Variables
pushups = LpVariable('pushups', lowBound=0, upBound=None, cat='Continuous')
running = LpVariable('running', lowBound=0, upBound=None, cat='Continuous')

# Define Objective Function
model += 3 * pushups + 130 * running

# Define Constraints
model += 0.2 * pushups + 10 * running <= 10
model += pushups >= 0
model += running >= 0

model.solve()

status =  LpStatus[model.status]

print(status, model.objective.value())

for v in model.variables():
    print(v.name,"=", v.value())

Optimal 150.0
pushups = 50.0
running = 0.0


### Trying out lpSum
- Enables you to define objective function via a list rather than having to explicitly write out all the sums
- In this exercise you are making two types (premium and budget) of ice cream, using heavy cream, whole milk, and sugar. One version is a premium version containing more cream than your budget version. You are looking to find the mixture of ingredients that minimizes the total costs of ingredients.
- Two Python lists called prod_type and ingredient have been created for you, along with a dictionary var_dict containing the decision variables of the model. You can explore them in the console.
- Define the objective function using lpSum() with list comprehension and the information in the table above. Iterate over the product types and multiply the dictionary variable by the correct ingredient cost.

- Prices
    - Cream: \$1.5 
    - Milk: \$0.125 
    - Sugar: \$0.10 
    
```
prod_type=['premium', 'budget']
ingredient=['cream', 'milk', 'sugar']
var_dict=
 {('budget', 'cream'): cb,
 ('budget', 'milk'): mb,
 ('budget', 'sugar'): sb,
 ('premium', 'cream'): cp,
 ('premium', 'milk'): mp,
 ('premium', 'sugar'): sp}
```

In [3]:
# Define Objective Function
model += lpSum([1.5 * var_dict[(i, 'cream')] 
                + 0.125 * var_dict[(i, 'milk')] 
                + 0.1 * var_dict[(i, 'sugar')]
                
                # Iterate over product types
                for i in prod_type])

### Logistics planning problem
You are consulting for kitchen oven manufacturer helping to plan their logistics for next month. There are two warehouse locations (New York, and Atlanta), and four regional customer locations (East, South, Midwest, West). The expected demand next month for East it is 1,800, for South it is 1,200, for the Midwest it is 1,100, and for West it is 1000. The cost for shipping each of the warehouse locations to the regional customer's is listed in the table below. Your goal is to fulfill the regional demand at the lowest price.

| Customer  | New York  | Atlanta  |
|---|---|---|
| East  |  \$211 | \$232  |
| South  | \$232  | \$212  |
| Midwest  | \$240  | \$230  |
| West  | \$300  | \$280  |

Two Python dictionaries costs and var_dict have been created for you containing the costs and decision variables of the model. You can explore them in the console.

```
costs = {('Atlanta', 'East'): 232,
 ('Atlanta', 'Midwest'): 230,
 ('Atlanta', 'South'): 212,
 ('Atlanta', 'West'): 280,
 ('New York', 'East'): 211,
 ('New York', 'Midwest'): 240,
 ('New York', 'South'): 232,
 ('New York', 'West'): 300}
 
var_dict = {('Atlanta', 'East'): atle,
 ('Atlanta', 'Midwest'): atlm,
 ('Atlanta', 'South'): atls,
 ('Atlanta', 'West'): atlw,
 ('New York', 'East'): ne,
 ('New York', 'Midwest'): nm,
 ('New York', 'South'): ns,
 ('New York', 'West'): nw}
```

Our decision variables in this exercise are the number of shipments for each warehouse and customer combination. You want to minimize the total cost of shipping. The total cost of shipping is the shipping costs along a route multiplied by the number of shipments along that route.

In [ ]:
from pulp import *

# Initialize Model
model = LpProblem("Minimize Transportation Costs", LpMinimize)

# Build the lists and the demand dictionary
warehouse = ['New York', 'Atlanta']
customers = ['East', 'South', 'Midwest', 'West']
regional_demand = [1800, 1200, 1100, 1000]
demand = dict(zip(customers, regional_demand))

# Define Objective
model += lpSum([costs[(w, c)] * var_dict[(w, c)] 
                for c in customers for w in warehouse])

# For each customer, sum warehouse shipments and set equal to customer demand
for c in customers:
    model += lpSum([var_dict[(w, c)] for w in warehouse]) == demand[c]

### Logistics planning problem 2
You are again consulting for kitchen oven manufacturer helping to plan their logistics. This time you are attempting to put together a plan for the next six months (Jan.-Jun.). There are still two warehouse locations (New York, and Atlanta), and four regional customer locations (East, South, Midwest, West). The cost for shipping for each of the warehouse locations to the regional customer's is listed in the table below. Your goal is to determine the number of shipments from each warehouse to customers that provides the lowest costs.

A Python dictionary named, costs containing the costs of the model, and three lists months, warehouse, and customers have been created for you. costs has been printed for you, you can explore the lists in the console as well. Additionally, the model has been initialized for you.



| Customer  | New York  | Atlanta  |
|---|---|---|
| East  |  \$211 | \$232  |
| South  | \$232  | \$212  |
| Midwest  | \$240  | \$230  |
| West  | \$300  | \$280  |

```
months=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']
warehouse=['New York', 'Atlanta']
customers=['East', 'South', 'Midwest', 'West']
costs = {('Atlanta', 'East'): 232,
 ('Atlanta', 'Midwest'): 230,
 ('Atlanta', 'South'): 212,
 ('Atlanta', 'West'): 280,
 ('New York', 'East'): 211,
 ('New York', 'Midwest'): 240,
 ('New York', 'South'): 232,
 ('New York', 'West'): 300}

```

In [ ]:
# Define decision variables
key = [(m, w, c) for m in months for w in warehouse for c in customers]
var_dict = LpVariable.dicts('num_of_shipments', 
                            key, 
                            lowBound=0, cat='Integer')

# Use the LpVariable dictionary variable to define objective
model += lpSum([costs[(w, c)] * var_dict[(m, w, c)] 
                for m in months for w in warehouse for c in customers])

### Traveling salesman problem (TSP)
The Traveling Salesman Problem (TSP) is a popular problem and has applications is logistics. In the TSP a salesman is given a list of cities, and the distance between each pair. He is looking for the shortest route going from the origin through all points before going back to the origin city again. This is a computationally difficult problem to solve but Miller-Tucker-Zemlin (MTZ) showed it can be completed using Integer Linear Programing. In this exercise you are going to define the objective and some constraints for of the TSP for a small dataset with 15 cities (see the image below). Your goal is to try out using LpVariable.dicts with list comprehension.

Three Python variables n, cities, and dist have been created for you 1. The n variable is the number of cities, cities is a list of the cities numbered and dist is a pandas DataFrame with the pairwise distance between each city. You can explore them in the console. Additionally, the model has been initialized for you.

```
n=15
cities=range(0, 15)
```

Use LpVariable.dicts to create a dictionary x holding binary variables for each city to city pair, and to create a dictionary u holding an integer LpVariable for each city.

Define the objective function by summing together, the distance between each pair of cities multiplied by the binary decision variable x that you created earlier.

Use the x binary variable to define a set of constraints to requires that each city be arrived at from exactly one other city, and a second set of constraints that requires that from each city there is a departure to exactly one other city. In the first set of constraints if we loop through the list of destination cities we should sum over the departure cities and ensure they sum to 1. In the second set of constraints if we loop through the list of arrival cities we should sum over the departure cities and ensure they sum to 1.

In [ ]:
# Define Decision Variables
x = LpVariable.dicts('X', [(c1, c2) for c1 in cities for c2 in cities], 
                     cat='Binary')
u = LpVariable.dicts('U', [c1 for c1 in cities], 
                     lowBound=0, upBound=(n-1), cat='Integer')

In [ ]:
# Define Decision Variables
x = LpVariable.dicts('X', [(c1, c2) for c1 in cities for c2 in cities], 
                     cat='Binary')
u = LpVariable.dicts('U', [c1 for c1 in cities], 
                     lowBound=0, upBound=(n-1), cat='Integer')

# Define Objective
model += lpSum([dist.iloc[c1, c2] * x[(c1, c2)] 
                for c1 in cities for c2 in cities])

# Define Constraints
for c2 in cities:
    model += lpSum([x[(c1, c2)] for c1 in cities]) == 1
for c1 in cities:
    model += lpSum([x[(c1, c2)] for c2 in cities]) == 1

### Scheduling workers problem
You are looking to hire workers to work in a warehouse. Each worker is expected to work 5 consecutive days and then have two days off. The chart below has the estimated number of workers you will need each day. You are looking to hire the minimum number of workers to handle the workload for each day.

|  Day of Week |  Employees Needed |
|---|---|
| 0 = Monday  | 31  |
| 1 = Tuesday  | 45  |
| 2 = Wednesday  | 40  |
| 3 = Thursday  | 40  |
| 4 = Friday  | 48  |
| 5 = Saturday  | 30  |
| 6 = Sunday  | 25  |

The pulp module has been imported and the model has been initialized for you. Also, the variable days has been defined for you as a list of number from 0-6.

In [ ]:
# The class has been initialize, and x, days, objective function defined
model = LpProblem("Minimize Staffing", LpMinimize)
days = list(range(7))
x = LpVariable.dicts('staff_', days, lowBound=0, cat='Integer')
model += lpSum([x[i] for i in days])

# Define Constraints
model += x[0] + x[3] + x[4] + x[5] + x[6] >= 31
model += x[0] + x[1] + x[4] + x[5] + x[6] >= 45
model += x[0] + x[1] + x[2] + x[5] + x[6] >= 40
model += x[0] + x[1] + x[2] + x[3] + x[6] >= 40
model += x[0] + x[1] + x[2] + x[3] + x[4] >= 48
model += x[1] + x[2] + x[3] + x[4] + x[5] >= 30
model += x[2] + x[3] + x[4] + x[5] + x[6] >= 25

model.solve()

### Preventative maintenance scheduling

At a quarry they use diamond saws to cut slabs of marble. For preventative maintenance the saws are only allowed to run for 4 consecutive hours, afterwards a 1 hour inspection is completed before they are allowed to go back into service. The quarry operates 10-hour shifts. At the end of the shift if the saw blades have not been used for 4 consecutive hours the remaining time will be used at the start of the next shift. The expected number of saw blades needed for each hour is listed below. Our goal is to determine the minimum number of saw blades are needed for the shift.

Expected Workload - (Note that the chart at hour 0)	
    
| Hour  | Saws Needed  |
|---|---|
| 0  | 7  |
| 1  | 7  |
| 2  | 7  |
| 3  | 6  |
| 4  | 5  |
| 5 |  6 |
| 6  | 6  |
| 7  | 7  |
| 8  | 7  |
| 9  | 6  |

In [ ]:
# The class has been initialize, and x, hours and objective fuction defined
model = LpProblem("Minimize Staffing", LpMinimize)
hours = list(range(10))
x = LpVariable.dicts('saws_', hours, lowBound=0, cat='Integer')
model += lpSum([x[i] for i in hours])

# Define Constraints
model += x[0] + x[2] + x[3] + x[4] + x[5] + x[7] + x[8] + x[9] >= 7
model += x[0] + x[1] + x[3] + x[4] + x[5] + x[6] + x[8] + x[9] >= 7
model += x[0] + x[1] + x[2] + x[4] + x[5] + x[6] + x[7] + x[9] >= 7
model += x[0] + x[1] + x[2] + x[3] + x[5] + x[6] + x[7] + x[8] >= 6
model += x[1] + x[2] + x[3] + x[4] + x[6] + x[7] + x[8] + x[9] >= 5
model += x[0] + x[2] + x[3] + x[4] + x[5] + x[7] + x[8] + x[9] >= 6
model += x[0] + x[1] + x[3] + x[4] + x[5] + x[6] + x[8] + x[9] >= 6
model += x[0] + x[1] + x[2] + x[4] + x[5] + x[6] + x[7] + x[9] >= 7
model += x[0] + x[1] + x[2] + x[3] + x[5] + x[6] + x[7] + x[8] >= 7
model += x[1] + x[2] + x[3] + x[4] + x[6] + x[7] + x[8] + x[9] >= 6

model.solve()

### Decision variables of case study

Continue the case study of the Capacitated Plant Location model of a car manufacture. You are given four Pandas data frames demand, var_cost, fix_cost, and cap containing the regional demand (thous. of cars), variable production costs (thous. $US), fixed production costs (thous. $US), and production capacity (thous. of cars). All these variables have been printed to the console for your viewing.

Continue the case study of the Capacitated Plant Location model of a car manufacture. You are given four Pandas data frames demand, var_cost, fix_cost, and cap containing the regional demand (thous. of cars), variable production costs (thous. $US), fixed production costs (thous. $US), and production capacity (thous. of cars). Two python lists loc, and size have also been created, containing the different locations, and the two types of plant capacities. All these variables have been printed to the console for your viewing. The code to initialize, and defined the decision variables has been completed for you.

In [20]:
# Initialize, and Define Decision Vars.
model = LpProblem("Capacitated Plant Location Model", LpMinimize)
loc = ['USA', 'Germany', 'Japan', 'Brazil', 'India']
size = ['Low_Cap','High_Cap']
x = LpVariable.dicts("production_", [(i,j) for i in loc for j in loc],
                     lowBound=0, upBound=None, cat='Continous')
y = LpVariable.dicts("plant_", 
                     [(i,s) for s in size for i in loc], cat='Binary')

# Define objective function
model += (lpSum([fix_cost.loc[i,s] * y[(i,s)] 
                 for s in size for i in loc])
          + lpSum([var_cost.loc[i,j] * x[(i,j)] 
                   for i in loc for j in loc]))

### Logical constraint exercise
Your customer has ordered six products to be delivered over the next month. You will need to ship multiple truck loads to deliver all of the products. There is a weight limit on your trucks of 25,000 lbs. For cash flow reasons you desire to ship the most profitable combination of products that can fit on your truck.

| Product  | Weight (lbs)  | Profitability ($US)  |
|---|---|---|
| A  | 12,583  | 102,564  |
| B  | 9,204  | 130,043  |
| C  | 12,611  | 127,648  |
| D  | 12,131  | 155,058  |
| E  | 12,889  | 238,846  |
| F  | 11,529  | 197,030  |

Two Python dictionaries weight, and prof, and a list prod have been created for you containing the weight, profitability, and name of each product. You can explore them in the console.

```
weight={'A': 12583, 'B': 9204, 'C': 12611, 'D': 12131, 'E': 12889, 'F': 11529}
prof={'A': 102564, 'B': 130043, 'C': 127648, 'D': 155058, 'E': 238846, 'F': 197030}
prod=['A', 'B', 'C', 'D', 'E', 'F']
```

In [ ]:
# Initialized model, defined decision variables and objective
model = LpProblem("Loading Truck Problem", LpMaximize)
x = LpVariable.dicts('ship_', prod, cat='Binary')
model += lpSum([prof[i] * x[i] for i in prod])

# Define Constraint
model += lpSum([weight[i] * x[i] for i in prod]) <= 25000
model += x['D'] + x['E'] + x['F'] <= 1

model.solve()
for i in prod:
    print("{} status {}".format(i, x[i].varValue))

### Logical constraints exercise 2
You work at a trucking distribution center and you need to decide which of 6 customer locations you will send a truck to. Your goal is to minimize the distance a truck travels.

| Location  | Distance  |
|---|---|
| A  | 86  |
| B  | 95  |
| C  | 205  |
| D  | 229  |
| E  | 101  |
| F  | 209  |

A dictionary dist, and a list cust have been created for you containing the distance, and name of each customer location. These inputs have been printed in console for you.

```
dist={'A': 86, 'B': 95, 'C': 205, 'D': 229, 'E': 101, 'F': 209}
cust=['A', 'B', 'C', 'D', 'E', 'F']
```

In [ ]:
model = LpProblem("Loading Truck Problem", LpMinimize)
x = LpVariable.dicts('ship_', cust, cat='Binary')
model += lpSum([dist[i]*x[i] for i in cust])

# Define Constraint
model += x['A'] + x['B'] + x['C'] + x['D'] + x['E'] + x['F'] >= 1
model += x['A'] - x['D'] <= 0
model += x['B'] - x['E'] <= 0

model.solve()
for i in cust:
    print("{} status {}".format(i, x[i].varValue))

### Dependent demand constraint exercise
You are developing a production plan for 5 products (A through E). One unit of product E comprises 2 units of A, and 1 unit of C. Product C is also sold directly to customers. Your planning horizon is 3 months. You are looking to determine how much of each product to produce by month that meets the customer's demand and minimizes the total costs.

A Pandas DataFrame named demand is printed in the console and contains the monthly customer demand for each product. Additionally, the code of the PuLP model to initialize, define decision variables, objective function, and constraint so production is greater than or equal to demand has been given to you.

In [ ]:
# Initialize, Define Decision Vars., Objective Function, and Constraints
model = LpProblem("Aggregate Production Planning", LpMinimize)
time = [0, 1, 2]
prod = ['A', 'B', 'C', 'D', 'E']
X = LpVariable.dicts("prod", [(t, p) for p in prod for t in time], 
                     lowBound=0, cat="Integer")
model += lpSum([costs.loc[t, p]*X[(t, p)] for p in prod for t in time])
for p in prod:
    for t in time:
        model += X[(t, p)] >= demand.loc[t, p]
        
# Define Dependent Demand Constraints
for t in time:
    model += 2 * X[(t, 'E')] <= X[(t, 'A')]
    model += 1 * X[(t, 'E')] + demand.loc[t, 'C'] <= X[(t, 'C')]

### Constraint combination exercise
You are working on a distribution plan for a warehouse network. The network has two warehouses (W1, and W2) and each can ship three different types of products (A, B, and C). W1 is small and can either ship 10 products A per a week or 15 products B per a week or 20 products C per a week. You are looking to minimize the total costs.

A Pandas DataFrame named demand is printed in the console and contains the monthly demand for each product. Additionally, the code of the PuLP model to initialize, define decision variables, objective function, and constraint so total shipment of each product equals its demand.

In [ ]:
# Initialize, Define Decision Vars., Objective Function, and Constraints
model = LpProblem("Distribution Planning", LpMinimize)
wh = ['W1','W2']
prod = ['A', 'B', 'C']
X = LpVariable.dicts("ship", [(w, p, c) for c in cust for p in prod for w in wh], 
                     lowBound=0, cat="Integer")
model += lpSum([X[(w, p, c)]*costs.loc[(w, p), c]  for c in cust for p in prod for w in wh])
for c in cust:
    for p in prod:
        model += lpSum([X[(w, p, c)] for w in wh]) == demand.loc[p, c]

# Define Dependent Demand Constraints
model += ((1/10) * lpSum([X[('W1', 'A', c)] for c in cust]) 
          + (1/15) * lpSum([X[('W1', 'B', c)] for c in cust])
          + (1/20) * lpSum([X[('W1', 'C', c)] for c in cust])) <= 4

### Exercise
Exercise
Constraints of case study exercise
Continue the case study of the Capacitated Plant Location model of a car manufacture. You are given four Pandas data frames demand, var_cost, fix_cost, and cap containing the regional demand (thous. of cars), variable production costs (thous. $ US), fixed production costs (thous. $ US), and production capacity (thous. of cars). Two python lists loc, and size have also been created, containing the different locations, and the two types of plant capacities. All these variables have been printed to the console for your viewing. The code to initialize the decision variables, define them and create the objective function has been completed for you.

In [ ]:
# Initialize, Define Decision Vars., and Objective Function
model = LpProblem("Capacitated Plant Location Model", LpMinimize)
loc = ['USA', 'Germany', 'Japan', 'Brazil', 'India']
size = ['Low_Cap','High_Cap']
x = LpVariable.dicts("production_", [(i,j) for i in loc for j in loc],
                     lowBound=0, upBound=None, cat='Continuous')
y = LpVariable.dicts("plant_", 
                     [(i,s) for s in size for i in loc], cat='Binary')
model += (lpSum([fix_cost.loc[i,s] * y[(i,s)] for s in size for i in loc])
          + lpSum([var_cost.loc[i,j] * x[(i,j)] for i in loc for j in loc]))

# Define the constraints
for j in loc:
    model += lpSum([x[(i, j)] for i in loc]) == demand.loc[j,'Dmd']
for i in loc:
    model += lpSum([x[(i, j)] for j in loc]) <= lpSum([cap.loc[i,s] * y[(i,s)] 
                                                       for s in size])

### Adding logical constraint in case study exercise
Continue the case study of the Capacitated Plant Location model of a car manufacture. You are given four Pandas data frames demand, var_cost, fix_cost, and cap containing the regional demand (thous. of cars), variable production costs (thous. $US), fixed production costs (thous. $US), and production capacity (thous. of cars). Two python lists loc, and size have also been created, containing the different locations, and the two types of plant capacities. All these variables have been printed to the console for your viewing. The code to initialize, defined the decision variables, create the objective function, and constraints has been completed for you.

In [ ]:
# Initialize, Define Decision Vars., Object. Fun., and Constraints
model = LpProblem("Capacitated Plant Location Model", LpMinimize)
loc = ['USA', 'Germany', 'Japan', 'Brazil', 'India']
size = ['Low_Cap','High_Cap']
x = LpVariable.dicts("production_", [(i,j) for i in loc for j in loc],
                     lowBound=0, upBound=None, cat='Integer')
y = LpVariable.dicts("plant_", 
                     [(i,s) for s in size for i in loc], cat='Binary')
model += (lpSum([fix_cost.loc[i,s] * y[(i,s)] for s in size for i in loc])
          + lpSum([var_cost.loc[i,j] * x[(i,j)] for i in loc for j in loc]))
for j in loc:
    model += lpSum([x[(i, j)] for i in loc]) == demand.loc[j,'Dmd']
for i in loc:
    model += lpSum([x[(i, j)] for j in loc]) <= lpSum([cap.loc[i,s]*y[(i,s)] 
                                                       for s in size])

# Define logical constraint
model += y[('USA','High_Cap')] - y[('Germany','Low_Cap')] <= 0

### Solving production plan exercise
In this exercise you are working on a production plan for manufacturing kitchen cabinets over the next four months. There are setup, production, and inventory storing costs. You are given the starting inventory, the production capacity and the demand for cabinets for each month. Your goal is to determine how many cabinets should be produced, and the ending inventory in each period that minimizes the overall costs over the four months.

The PuLP model has been completed for you and stored in the variable model. The model’s decision variables include, x which equals the quantity of cabinets produced in a month, and s which shows the inventory at the end of a month. Additionally, a Python list of the different time periods named time has been created for you.

In [ ]:
# Print status
print(LpStatus[model.status])

# Print Objective Fun. Value
print("Total Cost = ", value(model.objective))

In [ ]:
# Write model specifications to a file
file_name = 'production_planning_lp.txt'
model.writeLP(file_name)
f = open(file_name)
print(f.read())
f.close()

In [ ]:
# Initialize, Define Decision Vars.
model = LpProblem("Production Planning", LpMinimize)
time = [1, 2, 3, 4]
s = LpVariable.dicts("stock", [0, 1, 2, 3, 4], lowBound=0, cat="Integer")
x = LpVariable.dicts("prod", time, lowBound=0, cat="Integer")
y = LpVariable.dicts("plant_on", time, lowBound=0, cat="Binary")

# Define Objective to Include Fixed Costs
model += lpSum([d.loc[t,"unit_prod"] * x[t] + d.loc[t,"unit_inv"]*s[t] 
                + d.loc[t,"fixed"] * y[t]
                for t in time])

### Sensitivity analysis exercise
You are doing the resource planning for a lawn furniture company. They manufacture decorative sets of legs for lawn chairs, benches, and tables from metal tubes using a two step process involving tube-bending, and welding. The profit the company receives from the sale of each product is $3 for a set of chair legs, $3 for a set of bench legs, and $5 for a set of table legs. You are trying to plan the production mix for the upcoming season. Unfortunately, due to a strike there is only 2000 lbs tubing available for production on-hand.

The time and raw material requirements for each product are printed in the console. Also, the PuLP model has been completed for you and stored in the variable model. The constraints of the model are printed to the console.

In [ ]:
# Solve Model, Print Status, Desision Var, and Obj. Value
model.solve()
print("Model Status: {}".format(pulp.LpStatus[model.status]))
for v in model.variables():
    print(v.name, "=", v.varValue)
print("Objective = ", value(model.objective))

# Print Shadow Price and Slack
o = [{'name': name, 'shadow price': c.pi, 'slack': c.slack} 
     for name, c in model.constraints.items()]
print(pd.DataFrame(o))

### Shadow price and slack exercise pt1
You are planning what cupcakes a bakery should make. The bakery can make either:

regular size cupcake: profit = $5
a jumbo cupcake twice the regular size: profit = $10
There are 2 constraints on oven hours (30) and worker hours (65). This scenario has been modeled in PuLP for you and a solution found. The model status, decision variables values, objective value (i.e. profit), the shadow prices and slack of the constraints have been printed in the shell.

The sample script is a copy of that code. You will adjust the constraints to see how the optimal solution changes.

In [ ]:
# Increase the 1st constraint to 31. Run the code and see the change in the objective 
# compared to the original solution.

# Define Constraints, Solve, Print Status, Variables, Objective
model = LpProblem("Maximize Bakery Profits", LpMaximize)
R = LpVariable('Regular_production', lowBound=0, cat='Continuous')
J = LpVariable('Jumbo_production', lowBound=0, cat='Continuous')
model += 5 * R + 10 * J, "Profit"

# Adjust the constraint
model += 0.5 * R + 1 * J <= 30
model += 1 * R + 2.5 * J <= ____

# Solve Model, Print Status, Variables, Objective, Shadow and Slack
model.solve()
print("Model Status: {}".format(pulp.LpStatus[model.status]))
for v in model.variables():
    print(v.name, "=", v.varValue)
print("Objective = $", value(model.objective))
o = [{'name':name, 'shadow price':c.pi, 'slack': c.slack} 
     for name, c in model.constraints.items()]
print(pd.DataFrame(o))

In [ ]:
# Increase the 2nd constraint to 80. Run the code and see no change 
# in the objective compared to the original solution.

# Define Constraints, Solve, Print Status, Variables, Objective
model = LpProblem("Maximize Bakery Profits", LpMaximize)
R = LpVariable('Regular_production', lowBound=0, cat='Continuous')
J = LpVariable('Jumbo_production', lowBound=0, cat='Continuous')
model += 5 * R + 10 * J, "Profit"

# Adjust the constraint
model += 0.5 * R + 1 * J <= 30
model += 1 * R + 2.5 * J <= 80

# Solve Model, Print Status, Variables, Objective, Shadow and Slack
model.solve()
print("Model Status: {}".format(pulp.LpStatus[model.status]))
for v in model.variables():
    print(v.name, "=", v.varValue)
print("Objective = $", value(model.objective))
o = [{'name':name, 'shadow price':c.pi, 'slack': c.slack} 
     for name, c in model.constraints.items()]
print(pd.DataFrame(o))

In [ ]:
# Decrease the 2nd constraint to 60. Run the code and see no 
# change in the objective compared to the original solution.

# Define Constraints, Solve, Print Status, Variables, Objective
model = LpProblem("Maximize Bakery Profits", LpMaximize)
R = LpVariable('Regular_production', lowBound=0, cat='Continuous')
J = LpVariable('Jumbo_production', lowBound=0, cat='Continuous')
model += 5 * R + 10 * J, "Profit"

# Adjust the constraint
model += 0.5 * R + 1 * J <= 30
model += 1 * R + 2.5 * J <= 60

# Solve Model, Print Status, Variables, Objective, Shadow and Slack
model.solve()
print("Model Status: {}".format(pulp.LpStatus[model.status]))
for v in model.variables():
    print(v.name, "=", v.varValue)
print("Objective = $", value(model.objective))
o = [{'name':name, 'shadow price':c.pi, 'slack': c.slack} 
     for name, c in model.constraints.items()]
print(pd.DataFrame(o))

In [ ]:
# Decrease the 2nd constraint to 59. Run the code and see a 
# change in the objective compared to the original solution.

# Define Constraints, Solve, Print Status, Variables, Objective
model = LpProblem("Maximize Bakery Profits", LpMaximize)
R = LpVariable('Regular_production', lowBound=0, cat='Continuous')
J = LpVariable('Jumbo_production', lowBound=0, cat='Continuous')
model += 5 * R + 10 * J, "Profit"

# Adjust the constraint
model += 0.5 * R + 1 * J <= 30
model += 1 * R + 2.5 * J <= 59

# Solve Model, Print Status, Variables, Objective, Shadow and Slack
model.solve()
print("Model Status: {}".format(pulp.LpStatus[model.status]))
for v in model.variables():
    print(v.name, "=", v.varValue)
print("Objective = $", value(model.objective))
o = [{'name':name, 'shadow price':c.pi, 'slack': c.slack} 
     for name, c in model.constraints.items()]
print(pd.DataFrame(o))

In Step 1, You notice that changing the right-hand-side of a constraint by 1 increases the objective value by the amount of the shadow price. In Step 2, you should have noticed that increasing the right-hand-side of a non-binding constraint will not change the optimal solution. In Step 3, you should have noticed that decreasing the right-hand-side of a non-binding constraint equal to slack will not change the optimal solution but make it a binding constraint. In Step 4, you should have noticed that increasing the right-hand-side by more than the slack with change the optimal solution.

### Shadow price and slack exercise pt2
In this exercise you are working on the production plan for a company over the next 4 months. Your goal is to determine how much should be produced to minimize the production (fixed + variable), and storage costs while meeting the customers demand. The are constraints on the production capacity and demand each month.

Complete the code, near the bottom of the sample code, to create a Pandas DataFrame that shows the slack of the constraints.

You need to loop through the model constraints and call the constraint slack attribute to create the list of Python dictionaries.

In [ ]:
model = LpProblem("Production Planning", LpMinimize)
time = [1, 2, 3, 4]
s = LpVariable.dicts("stock_in", [0, 1, 2, 3, 4], lowBound=0, cat="Integer")
x = LpVariable.dicts("prod_in", time, lowBound=0, cat="Integer")
y = LpVariable.dicts("plant_on_", time, lowBound=0, cat="Binary")
model += lpSum([d.loc[t,"unit_prod"]*x[t] + d.loc[t,"unit_inv"]*s[t] 
                + d.loc[t,"fixed_setup"]*y[t] for t in time])
s[0] = 100
for t in time:
    model += s[t-1] + x[t] == d.loc[t,"demand"] + s[t]
    model += x[t] <= d.loc[t,"prod_cap"]*y[t]
model.solve()

# Print the Constraint Slack
o = [{'name':name, 'slack':c.slack} 
     for name, c in model.constraints.items()]
print(pd.DataFrame(o))

### Solving the model case study exercise
Continue the case study of the Capacitated Plant Location model of a car manufacture. The PuLP model has been completed and solved for you. It is stored in the variable model.

The decision variables x and y respectively represent the production quantities of the different regions, and if a production plant is opened. Additionally, two python lists loc and size have also been created, containing the different locations, and the two types of plant capacities. Finally, the input data for the model has been printed to the console for reference.

- Print the values of how much is produced and shipped from one region to another.
- Next, print the status value of the different regional plants of low and high capacity.
- Finally, print the objective value.

In [ ]:
# Print the Production Quantities
o = [{'prod':'{} to {}'.format(i,j), 'quantity':x[(i,j)].varValue} 
     for i in loc for j in loc]
print(pd.DataFrame(o))

# Print the Plant Values of the different regions
o = [{'lowCap':y[(i,size[0])].varValue, 'highCap':y[(i,size[1])].varValue}
     for i in loc]
print(pd.DataFrame(o, index=loc))

# Print the Objective Value
print('Objective = ', value(model.objective))

### Simulation testing solution exercise
You are working to minimize the transportation costs of shipping products from warehouses in New York, and Atlanta, to customers in the East, South, Midwest, and West regions of the US. The input data of transportation costs and demand for each region is given to you in the variable d which is printed in the console.

A function named run_pulp_model() has been created for you to solve and return results of the PuLP model using the random objective from step 1. Complete the code to call the function 100 times, saving the results in a list.

In [ ]:
# Add noise to the objective function
model += lpSum([(X[(c, w)] + (d.loc[c, w] + random.normalvariate(0, 5)))
                for w in wh for c in cust])

# Call the function 100 times and append results to list
output = []
for i in range(100):
    output.append(run_pulp_model(d))
df = pd.DataFrame(output)

# Print the value count df series Midwest from Atlanta
print(df['Midwest from Atlanta'].value_counts())

# Print the value count df series East from Atlanta
print(df['East from Atlanta'].value_counts())

### Simulation testing capacitated model
Continue the case study of the Capacitated Plant Location model of a car manufacture. The PuLP model has been completed and solved for you. It is stored in the variable model. The decision variables x, and y respectively represent the production quantities of the different regions, and if a production plant is opened. Additionally, two python lists loc, and size have also been created, containing the different locations, and the two types of plant capacities. Finally, the input data for the model has been printed to the console for you for reference.

Complete the code of the objective function that uses the random.normalvariate() method to add noise to the variable costs estimates, centered at 0 (thousands of $US dollars) with a standard deviation of 5 (thousands of $US dollars).

Add noise to the regional demand estimates. Set the parameters of the random.normalvariate() method where the mean=0 and the standard deviation is 5% of the demand estimate.

A function named run_pulp_model(fix_cost, var_cost, demand, cap) has been created for you to solve and return results of the PuLP model for simulation testing. Complete the code to call the function 100 times, saving the results in a list.

A function named run_pulp_model(fix_cost, var_cost, demand, cap) has been created for you to solve and return results of the PuLP model for simulation testing. Complete the code to call the function 100 times, saving the results in a list.

Plot the histogram of the production in Brazil (this could take a bit longer than usual to run).

In [ ]:
# Define Objective
model += (lpSum([fix_cost.loc[i,s] * y[(i,s)] for s in size for i in loc])
          + lpSum([(var_cost.loc[i,j] + random.normalvariate(0, 5))*x[(i,j)] 
                   for i in loc for j in loc]))

# Define the Constraints
for j in loc:
    rd = random.normalvariate(0, demand.loc[j,'Dmd'] * 0.05)
    model += lpSum([x[(i, j)] for i in loc]) == (demand.loc[j,'Dmd'] + rd)
    
# Call the function 100 times and append results to list
output = []
for i in range(100):
    output.append(run_pulp_model(fix_cost, var_cost, demand, cap))
df = pd.DataFrame(output)

# Import matplotlib
import matplotlib.pyplot as plt

# Call the function 100 times and append results to list
output = []
for i in range(100):
    output.append(run_pulp_model(fix_cost, var_cost, demand, cap))
df = pd.DataFrame(output)

# Plot histogram of Brazil Production
plt.hist(df['Brazil'])
plt.title('Histogram of Prod. In Brazil Region')
plt.show()

In [ ]:
# Import matplotlib.pyplot
import matplotlib.pyplot as plt
plt.title('Histogram of Prod. At USA Region')

# Histogram of USA production
plt.hist(df['USA'])
plt.show()
plt.title('Histogram of Prod. At Japan Region')

# Histogram of Japan production
plt.hist(df['Japan'])
plt.show()